In [1]:
!pip install h5py
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 30.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 69.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 84.1 MB/s eta 0:00:00a 0:00:01


In [2]:
import h5py
import torch
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch.optim import AdamW
from tqdm import tqdm
from torchvision.transforms import ToTensor, ToPILImage
from torchvision.utils import save_image
from PIL import Image
import os
import numpy as np
from sklearn.model_selection import train_test_split

from torchvision import transforms

class RainDataset(Dataset):
    def __init__(self, input_paths, target_paths):
        super().__init__()
        self.input_paths = input_paths
        self.target_paths = target_paths
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((256, 256)),
        ])
        
    def __len__(self):
        return len(self.input_paths)
    
    def __getitem__(self, idx):
        input_image = Image.open(self.input_paths[idx]).convert("RGB")
        target_image = Image.open(self.target_paths[idx]).convert("RGB")
        
        input_image = self.transform(input_image)
        target_image = self.transform(target_image)
        return input_image, target_image


In [3]:
import torch
from torch import nn

class GeneratorLoss(nn.Module):
    def __init__(self, alpha=100):
        super().__init__()
        self.alpha=alpha
        self.bce=nn.BCEWithLogitsLoss()
        self.l1=nn.L1Loss()
        
    def forward(self, fake, real, fake_pred):
        fake_target = torch.ones_like(fake_pred)
        loss = self.bce(fake_pred, fake_target) + self.alpha* self.l1(fake, real)
        return loss
    
    
class DiscriminatorLoss(nn.Module):
    def __init__(self,):
        super().__init__()
        self.loss_fn = nn.BCEWithLogitsLoss()
        
    def forward(self, fake_pred, real_pred):
        fake_target = torch.zeros_like(fake_pred)
        real_target = torch.ones_like(real_pred)
        fake_loss = self.loss_fn(fake_pred, fake_target)
        real_loss = self.loss_fn(real_pred, real_target)
        loss = (fake_loss + real_loss)/2
        return loss

In [4]:
import torch
from torch import nn
from torch.nn import functional as F

        
class BasicBlock(nn.Module):
    """Basic block"""
    def __init__(self, inplanes, outplanes, kernel_size=4, stride=2, padding=1, norm=True):
        super().__init__()
        self.conv = nn.Conv2d(inplanes, outplanes, kernel_size, stride, padding)
        self.isn = None
        if norm:
            self.isn = nn.InstanceNorm2d(outplanes)
        self.lrelu = nn.LeakyReLU(0.2, inplace=True)
        
    def forward(self, x):
        fx = self.conv(x)
        
        if self.isn is not None:
            fx = self.isn(fx)
            
        fx = self.lrelu(fx)
        return fx
    
    
class Discriminator(nn.Module):
    """Basic Discriminator"""
    def __init__(self,):
        super().__init__()
        self.block1 = BasicBlock(3, 64, norm=False)
        self.block2 = BasicBlock(64, 128)
        self.block3 = BasicBlock(128, 256)
        self.block4 = BasicBlock(256, 512)
        self.block5 = nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)
        
    def forward(self, x):
        fx = self.block1(x)
        fx = self.block2(fx)
        fx = self.block3(fx)
        fx = self.block4(fx)
        fx = self.block5(fx)
        
        return fx
    
    
class ConditionalDiscriminator(nn.Module):
    """Conditional Discriminator"""
    def __init__(self,):
        super().__init__()
        self.block1 = BasicBlock(6, 64, norm=False)
        self.block2 = BasicBlock(64, 128)
        self.block3 = BasicBlock(128, 256)
        self.block4 = BasicBlock(256, 512)
        self.block5 = nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)
        
    def forward(self, x, cond):
        x = torch.cat([x, cond], dim=1)

        fx = self.block1(x)
        fx = self.block2(fx)
        fx = self.block3(fx)
        fx = self.block4(fx)
        fx = self.block5(fx)
        
        return fx

In [5]:
import torch
from torch import nn
from torch.nn import functional as F


class EncoderBlock(nn.Module):
    """Encoder block"""
    def __init__(self, inplanes, outplanes, kernel_size=4, stride=2, padding=1, norm=True):
        super().__init__()
        self.lrelu = nn.LeakyReLU(0.2, inplace=True)
        self.conv = nn.Conv2d(inplanes, outplanes, kernel_size, stride, padding)
        
        self.bn=None
        if norm:
            self.bn = nn.BatchNorm2d(outplanes)
        
    def forward(self, x):
        fx = self.lrelu(x)
        fx = self.conv(fx)
        
        if self.bn is not None:
            fx = self.bn(fx)
            
        return fx

    
class DecoderBlock(nn.Module):
    """Decoder block"""
    def __init__(self, inplanes, outplanes, kernel_size=4, stride=2, padding=1, dropout=False):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.deconv = nn.ConvTranspose2d(inplanes, outplanes, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(outplanes)       
        
        self.dropout=None
        if dropout:
            self.dropout = nn.Dropout2d(p=0.5, inplace=True)
            
    def forward(self, x):
        fx = self.relu(x)
        fx = self.deconv(fx)
        fx = self.bn(fx)

        if self.dropout is not None:
            fx = self.dropout(fx)
            
        return fx

    
class Generator(nn.Module):
    """Encoder-Decoder model"""
    def __init__(self,):
        super().__init__()
        
        self.encoder1 = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1)
        self.encoder2 = EncoderBlock(64, 128)
        self.encoder3 = EncoderBlock(128, 256)
        self.encoder4 = EncoderBlock(256, 512)
        self.encoder5 = EncoderBlock(512, 512)
        self.encoder6 = EncoderBlock(512, 512)
        self.encoder7 = EncoderBlock(512, 512)
        self.encoder8 = EncoderBlock(512, 512, norm=False)
        
        self.decoder8 = DecoderBlock(512, 512, dropout=True)
        self.decoder7 = DecoderBlock(512, 512, dropout=True)
        self.decoder6 = DecoderBlock(512, 512, dropout=True)
        self.decoder5 = DecoderBlock(512, 512)
        self.decoder4 = DecoderBlock(512, 256)
        self.decoder3 = DecoderBlock(256, 128)
        self.decoder2 = DecoderBlock(128, 64)
        self.decoder1 = nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1)
        
    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        e5 = self.encoder5(e4)
        e6 = self.encoder6(e5)
        e7 = self.encoder7(e6)
        e8 = self.encoder8(e7)

        d8 = self.decoder8(e8)
        d7 = self.decoder7(d8)
        d6 = self.decoder6(d7)
        d5 = self.decoder5(d6)
        d4 = self.decoder4(d5)
        d3 = self.decoder3(d4)
        d2 = F.relu(self.decoder2(d3))
        d1 = self.decoder1(d2)
        
        return torch.tanh(d1)
    
    
class UnetGenerator(nn.Module):
    """Unet-like Encoder-Decoder model"""
    def __init__(self,):
        super().__init__()
        
        self.encoder1 = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1)
        self.encoder2 = EncoderBlock(64, 128)
        self.encoder3 = EncoderBlock(128, 256)
        self.encoder4 = EncoderBlock(256, 512)
        self.encoder5 = EncoderBlock(512, 512)
        self.encoder6 = EncoderBlock(512, 512)
        self.encoder7 = EncoderBlock(512, 512)
        self.encoder8 = EncoderBlock(512, 512, norm=False)
        
        self.decoder8 = DecoderBlock(512, 512, dropout=True)
        self.decoder7 = DecoderBlock(2*512, 512, dropout=True)
        self.decoder6 = DecoderBlock(2*512, 512, dropout=True)
        self.decoder5 = DecoderBlock(2*512, 512)
        self.decoder4 = DecoderBlock(2*512, 256)
        self.decoder3 = DecoderBlock(2*256, 128)
        self.decoder2 = DecoderBlock(2*128, 64)
        self.decoder1 = nn.ConvTranspose2d(2*64, 3, kernel_size=4, stride=2, padding=1)
        
    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        e5 = self.encoder5(e4)
        e6 = self.encoder6(e5)
        e7 = self.encoder7(e6)
        e8 = self.encoder8(e7)

        d8 = self.decoder8(e8)
        d8 = torch.cat([d8, e7], dim=1)
        d7 = self.decoder7(d8)
        d7 = torch.cat([d7, e6], dim=1)
        d6 = self.decoder6(d7)
        d6 = torch.cat([d6, e5], dim=1)
        d5 = self.decoder5(d6)
        d5 = torch.cat([d5, e4], dim=1)
        d4 = self.decoder4(d5)
        d4 = torch.cat([d4, e3], dim=1)
        d3 = self.decoder3(d4)
        d3 = torch.cat([d3, e2], dim=1)
        d2 = F.relu(self.decoder2(d3))
        d2 = torch.cat([d2, e1], dim=1)
        d1 = self.decoder1(d2)
        
        return torch.tanh(d1)

In [6]:
# ============================
# Helper functions
# ============================
def psnr(x, y, data_range=1.0):
    mse = torch.mean((x - y) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * torch.log10(data_range / torch.sqrt(mse))


def ssim(x, y, kernel_size=11, kernel_sigma=1.5, data_range=1.0, k1=0.01, k2=0.03):
    coords = torch.arange(kernel_size, dtype=x.dtype, device=x.device)
    coords -= (kernel_size - 1) / 2.0
    g = coords ** 2
    g = (- (g.unsqueeze(0) + g.unsqueeze(1)) / (2 * kernel_sigma ** 2)).exp()
    g /= g.sum()
    kernel = g.unsqueeze(0).repeat(x.size(1), 1, 1, 1)

    c1, c2 = (k1 * data_range) ** 2, (k2 * data_range) ** 2
    n_channels = x.size(1)
    mu_x = F.conv2d(x, weight=kernel, stride=1, padding=0, groups=n_channels)
    mu_y = F.conv2d(y, weight=kernel, stride=1, padding=0, groups=n_channels)

    mu_xx, mu_yy, mu_xy = mu_x ** 2, mu_y ** 2, mu_x * mu_y
    sigma_xx = F.conv2d(x ** 2, weight=kernel, stride=1, padding=0, groups=n_channels) - mu_xx
    sigma_yy = F.conv2d(y ** 2, weight=kernel, stride=1, padding=0, groups=n_channels) - mu_yy
    sigma_xy = F.conv2d(x * y, weight=kernel, stride=1, padding=0, groups=n_channels) - mu_xy

    cs = (2.0 * sigma_xy + c2) / (sigma_xx + sigma_yy + c2)
    ss = (2.0 * mu_xy + c1) / (mu_xx + mu_yy + c1) * cs
    return ss.mean()

In [7]:
def train_model(generator, discriminator, dataloader, gen_loss_fn, disc_loss_fn, optimizer_G, optimizer_D, device):
    generator.train()
    discriminator.train()
    
    total_gen_loss = 0.0
    total_disc_loss = 0.0
    num_batches = len(dataloader)
    
    for inputs, targets in tqdm(dataloader, desc="Training"):
        inputs, targets = inputs.to(device), targets.to(device)
        
        fake_images = generator(inputs)
        
        optimizer_D.zero_grad()
        real_preds = discriminator(targets, inputs)
        fake_preds = discriminator(fake_images.detach(), inputs)
        disc_loss = disc_loss_fn(fake_preds, real_preds)
        disc_loss.backward()
        optimizer_D.step()
        
        optimizer_G.zero_grad()
        fake_preds = discriminator(fake_images, inputs)
        gen_loss = gen_loss_fn(fake_images, targets, fake_preds)
        gen_loss.backward()
        optimizer_G.step()
        
        total_gen_loss += gen_loss.item()
        total_disc_loss += disc_loss.item()
    
    avg_gen_loss = total_gen_loss / num_batches
    avg_disc_loss = total_disc_loss / num_batches
    
    print(f"Avg Generator Loss: {avg_gen_loss:.4f}, Avg Discriminator Loss: {avg_disc_loss:.4f}")
    return avg_gen_loss, avg_disc_loss

def evaluate_model(generator, dataloader, device):
    generator.eval()
    total_l1_loss = 0.0
    total_psnr = 0.0
    total_ssim = 0.0
    l1_loss_fn = torch.nn.L1Loss()
    count=0

    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Evaluating"):
            inputs, targets = inputs.to(device), targets.to(device)
            
            fake_images = torch.clamp(generator(inputs), 0, 1)

            total_l1_loss += l1_loss_fn(fake_images, targets).item()

            total_psnr += psnr(fake_images, targets)
            total_ssim += ssim(fake_images, targets)
            count += 1

    avg_l1_loss = total_l1_loss / count
    avg_psnr = total_psnr / count
    avg_ssim = total_ssim / count

    print(f"Avg L1 Loss: {avg_l1_loss:.4f}, Avg PSNR: {avg_psnr:.2f}, Avg SSIM: {avg_ssim:.4f}")
    return avg_l1_loss, avg_psnr, avg_ssim

In [8]:
data_path = '/train/train/Rain13K'
test_size = 0.2
learning_rate = 1e-4
device = "cuda" if torch.cuda.is_available() else "cpu"

input_folder = os.path.join(data_path, 'input')
target_folder = os.path.join(data_path, 'target')

input_files = sorted([os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])
target_files = sorted([os.path.join(target_folder, f) for f in os.listdir(target_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])
    
train_inputs, test_inputs, train_targets, test_targets = train_test_split(
    input_files, target_files, test_size=test_size, random_state=42
)

train_dataset = RainDataset(train_inputs, train_targets)
test_dataset = RainDataset(test_inputs, test_targets)

In [ ]:
import os
from itertools import product
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch

batch_sizes = [32, 16, 8, 4, 1]
alpha_values = [300]
epochs = 50
save_epochs = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

base_result_dir = "result"

if not os.path.exists(base_result_dir):
    os.makedirs(base_result_dir)

for batch_size, alpha in product(batch_sizes, alpha_values):
    print(f"Testing Batch Size: {batch_size}, Alpha: {alpha}")
    
    batch_alpha_dir = os.path.join(base_result_dir, f"bs{batch_size}_alpha{alpha}")
    if not os.path.exists(batch_alpha_dir):
        os.makedirs(batch_alpha_dir)
    
    log_file = os.path.join(batch_alpha_dir, "gridsearch_log.txt")

    with open(log_file, "w") as f:
        f.write("Batch Size, Alpha, Epoch, Avg Gen Loss, Avg Disc Loss, Avg L1 Loss, Avg PSNR, Avg SSIM\n")
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    generator = UnetGenerator().to(device)
    discriminator = ConditionalDiscriminator().to(device)
    
    gen_loss_fn = GeneratorLoss(alpha=alpha).to(device)
    disc_loss_fn = DiscriminatorLoss().to(device)
    
    optimizer_G = AdamW(generator.parameters(), lr=learning_rate, betas=(0.9, 0.999))
    optimizer_D = AdamW(discriminator.parameters(), lr=learning_rate, betas=(0.9, 0.999))
    
    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}/{epochs}")

        avg_gen_loss, avg_disc_loss = train_model(generator, discriminator, train_loader, gen_loss_fn, disc_loss_fn, optimizer_G, optimizer_D, device)

        avg_l1_loss, avg_psnr, avg_ssim = evaluate_model(generator, test_loader, device)
 
        with open(log_file, "a") as f:
            f.write(f"{batch_size}, {alpha}, {epoch}, {avg_gen_loss:.4f}, {avg_disc_loss:.4f}, {avg_l1_loss:.4f}, {avg_psnr:.4f}, {avg_ssim:.4f}\n")

        if epoch in save_epochs:
            gen_save_path = os.path.join(batch_alpha_dir, f"generator_bs{batch_size}_alpha{alpha}_epoch{epoch}.pth")
            disc_save_path = os.path.join(batch_alpha_dir, f"discriminator_bs{batch_size}_alpha{alpha}_epoch{epoch}.pth")
            torch.save(generator.state_dict(), gen_save_path)
            torch.save(discriminator.state_dict(), disc_save_path)
            print(f"Models saved for Batch Size: {batch_size}, Alpha: {alpha}, Epoch: {epoch}")

Testing Batch Size: 32, Alpha: 300
Epoch 1/50



raining: 100%|██████████| 343/343 [01:18<00:00,  4.38it/s]

Avg Generator Loss: 30.8466, Avg Discriminator Loss: 0.4232



valuating: 100%|██████████| 86/86 [00:10<00:00,  8.03it/s]

Avg L1 Loss: 0.0677, Avg PSNR: 20.78, Avg SSIM: 0.5105
Models saved for Batch Size: 32, Alpha: 300, Epoch: 1
Epoch 2/50


...


raining: 100%|██████████| 10968/10968 [04:16<00:00, 42.73it/s]

Avg Generator Loss: 16.3500, Avg Discriminator Loss: 0.1703



valuating: 100%|██████████| 2743/2743 [00:15<00:00, 174.43it/s]

Avg L1 Loss: 0.0565, Avg PSNR: 24.61, Avg SSIM: 0.7501
Epoch 7/50


Training:  11%|█         | 1185/10968 [00:29<03:48, 42.83it/s]